In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

#temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), width=800))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Introduction the problem:**

The objective is to use the dataset market data to rank the stocks from highest to lowest expected returns and use the rank to predict the top 200 stocks and bottom 200 stocks.

The biggest winners in the stock market are those who are able to identify solid under valued investments. One notably investor is Carl Icahn. He had a business model that involved taking a large stake in underbought companies he believed to be undervalued. He would make large profits by identifying and buying solid under valued stocks and sale his position after they became overvalued. He was labeled as a successful investor on Wall Street and a hostile activist shareholder because of his investment strategies and takeovers.

In this analysis we will explore quantitative trading where predictions can be made from trained models. The financial data includes stock information and historical stock prices that can be analyzed. With over 2,000 stock data points we can analyze returns and movements. This will allow us rank by sectors.

Lets first start small by analyzing one stock then build from there.

**Introduce the Data:**

The Data-set is from the Kaggle competition: JPX Tokyo Stock Exange Predict. "https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/data"

The competition is described below: "Description import"

The data includes stock_list and different csv files:

- financials.csv
- options.csv
- secondary_stocks_prices.csv
- stock_prices.csv
- trads.csv

Lets first load our data and do some manipulation in pandas to prepare it in a readable format.

The Data-set contains quantitive data of 2,000 commonly traded stocks and options in the Japanese stock market. Some of the stock names include KYOKUYO CO, NIKKO EXCHANGE, and FIT Corporation from the years 2017 to 2021. Each security can be identified by a SecuritiesCode

The columns included in the stock_price csv file:


RowId: Unique ID of price records, the combination of Date and SecuritiesCode.
Date: Trade date
SecuritiesCode: Local securities code
Open: First traded price on a day in JPY

High: Highest traded price on a day in JPY

Low: Lowest traded price on a day in JPY

Close: Last traded price on a day in JPY

Volume: Number of traded stocks on a day
AdjustmentFactor: Used to calculate price when split

ExpectedDividend: Expected dividend value

Target: Change ratio

In [ ]:
sample = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv")
sample

In [ ]:
sample.nunique()
sample_date = sample.Date.unique()

In [ ]:
stock_prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
stock_prices

In [ ]:
stock_prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
stock_list = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")

stock_prices["Date"] = pd.to_datetime(stock_prices["Date"])
#stock_list["Name"] = pd.DataFrame(stock_list["Name"])

mcd_info = stock_list[stock_list["Name"] == "McDonald's Holdings Company(Japan),Ltd."]
mcd_code = mcd_info["SecuritiesCode"].values[0]
print("Mcdonalds Securities code: ",mcd_code)


tmpdf = stock_prices[stock_prices["SecuritiesCode"]==2702].reset_index(drop=True)
tmpdf.head(3)

In [ ]:
#Currency in JPY to covert to USD to compare close amount
exchange_rate = .0068 # 1 USD to 146.78 JPY in September
jpy_at_close = tmpdf["Close"].values[0]
convert_usd = jpy_at_close * exchange_rate
print("Price is JPY", jpy_at_close, "Price in USD", convert_usd)

In [ ]:

tmpdf = stock_prices[stock_prices["SecuritiesCode"] == 2702].reset_index(drop=True)

rate = 0.0068

# Function created
def jpy_to_usd(jpy, exchange_rate):
    return jpy * exchange_rate

# Columns to convert
columns_to_convert = ["Open", "High", "Low", "Close"]

# Convert JPY to USD
for column in columns_to_convert:
    tmpdf[column] = tmpdf[column].apply(lambda x: jpy_to_usd(x, rate))

tmpdf.head(3)

We can visualise the data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), width=800))

# Filter 'train' DataFrame for McDonald's stock (SecuritiesCode 2702)
mcdonalds_train = stock_prices[stock_prices["SecuritiesCode"] == 2702]

mcdonalds_train['Close'] = mcdonalds_train['Close'].apply(lambda x: jpy_to_usd(x, rate))

# Calculate
close_avg = mcdonalds_train.groupby('Date')['Close'].mean().rename('Closing Price (USD)')
returns = mcdonalds_train.groupby('Date')['Target'].mean().mul(100).rename('Average Return')
#close_avg = mcdonalds_train.groupby('Date')['Close'].mean().rename('Closing Price')
vol_avg = mcdonalds_train.groupby('Date')['Volume'].mean().rename('Volume')

# subplot
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

# colors 
colors = ['blue', 'green', 'red']

# Plot the metrics for McDonald's stock
for i, j in enumerate([returns, close_avg, vol_avg]):
    fig.add_trace(go.Scatter(x=sample_date, y=j, mode='lines',
                             name=j.name, marker_color=colors[i]), row=i+1, col=1)

# Update x-axis settings
fig.update_xaxes(rangeslider_visible=False,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=2, label="2y", step="year", stepmode="backward"),
                         dict(step="all")])),
                 row=1, col=1)

# Update layout
fig.update_layout(template=temp, title='McDonald\'s Stock Metrics',
                  hovermode='x unified', height=700,
                  yaxis1=dict(title='Stock Return', ticksuffix='%'),
                  yaxis2_title='Closing Price', yaxis3_title='Shares Traded',
                  showlegend=False)

# Show the plot
fig.show()

In [ ]:
tmpdf['Year'] = tmpdf['Date'].dt.year

# Create dictionary 
years = {year: tmpdf[tmpdf['Year'] == year]['Target'].mean() * 100 for year in tmpdf['Year'].unique()}

# Sort 
years = {k: v for k, v in sorted(years.items(), key=lambda item: item[0])}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(years.items()), columns=['Year', 'Avg_return'])


fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Bar(x=df['Year'], y=df['Avg_return'], text=df['Avg_return'].apply(lambda x: f'{x:.2f}%'),
                     textposition='auto', hovertemplate='Year: %{x}<br>Avg Return: %{y:.2f}%',
                     marker=dict(color='blue', opacity=0.7)))

fig.update_layout(template=temp, title='Yearly Average Stock Returns (McDonald\'s)',
                  xaxis_title='Year', yaxis_title='Average Return (%)', showlegend=False)

fig.show()

Sharpe Ratio is used to evaluate the daily spread of returns. Investopedia explains the Sharpe Ratio as a mathematical expression the measures risk and volatility. 

The name comes from Economist William Sharpe in 1966 who called it a reward-to-variability ratio.

The formula and calculation of the Sharpe Ratio:
Return - Risk Free Rate/Theta

![image.png](attachment:fe146870-b396-4619-bc71-d71a0956426a.png)

![image.png](attachment:2e8ef26e-ef1b-4bd4-b1d6-e8753bdea918.png)

We are looking for something that has a small denominator compared to its numerator. This type of analysis is better when comparing stocks to its peers.

The denominator is calculated by:

1. Take the return variance(How far each number in a data set is from the average) from the average return in each of the incremental periods, square it, and sum the squares from all the incremental periods.

2. Divide the sum by the number of incremental time periods.

3. Take a square root of the quotient.

The Sharpe Ratio can tell us the risk-adjusted relative return. It can compare a stocks historical amount relating to its benchmark to the expected variablity of the return.
In other words the Sharpe Ratio compares reward to the risk. 

If we wanted to find the Sharpe Ratio for Mcdonalds holdings we would first find the average annual return and then compare it to a low-risk investment like a government bond or ETF. We would subtract rate of the bond from Mcdonalds stock rate and then divide it by the Standard Deviation rate of Mcdonalds stock. Sharpe Ratios above 1 are considered "good" and offers excess returns compared to its volatility. However investors usually compare stocks in a portfolio with those in the market sector. So if Mcdonalds is in a ETF it would be compared with stocks in the Consumer Discretionary sector in USA, but this is not the same sector in Japan.

Lets use cardinality and correlation to eliminate unuseful columns

- column RowID is in string format is not useful as SecurityCode. We can get the same information by calling this column.

- column Date is useful since we need this to find the deviation

- Price columns (High, Low, Open and Close) are useful but is in JPX not USD. It is also missing 7608 values. We will need to replace this by comparing it to the Volume column.

- column Volue should have Nan for non trading days such as weekends. The missing values are related to the non trading days and can be replaced with 0 or NaN

When Analyzing the data we should check for the number of NaN values for each column and if the Volume column contains the same number of 0's as the number of missing values in the Close column.

We need to find a way to augment the data and replace values for the empty rows to fill Open, High, Low and Close.

pct_change()

Pandas has a funtion that calculates the percentage of change between elements in a row from the previous row.

Keep in mind volatility represents how large the stock prices swing around the average price. Less volitale means that the price is expected to stay around the average. There are several ways to measure volatility such as option pricing model and standard deviations of returns. 

We also need to know the moving average which is the rolling mean or movin mean. It is the moving average that is used in time-series to capture the short-term swing while keeping up with the trends. The average can measure swings in seconds,minutes, hours and days or any selected time-frame. For our analysis we want to measure days and periods.

Takeaways: A strategy to rank the stock you can use the Sharpe Ratio and set a buy vs sell alert. The alert will get triggered if the ratio is above 1.5 and also if the price is below the historical close price.

It will also be beneficial to keep track of trends and entry/exit timing. The trend can measure the current price relatice to the average high and low over the previous periods. The entry/exit measure the stock volume of buying and selling pressure. We can set a indicater to represent if the stock is oversold or overbought.

There was also some important things to consider such as the Equity Trading System failure on Oct 1, 2020 and other system failures for missing data

Lets check and see how many stock appear in the set and compare to see if any have missing records

In [ ]:
stock_prices['SecuritiesCode'].value_counts()

We see that stock 1301 has a 1202 records and 4169 has 232 records. This is a huge difference, showing that 1000 dates are missing.

In [ ]:
print(stock_prices.columns)

In [ ]:
print(stock_prices.SecuritiesCode)

Sharpe Ratio is good formula but considers movement in either direction risky. In other words the Sharpe ratio creates a curve that the stock should follow and gives it a score or rank based off the closeness to that curve. Even if the price is above the curve then the rank can be skewed. Risk is assessed only off a deviation of the standard. It treats positive and negitive returns equally. Another fact about the Sharpe Ratio is uses a linear relationship meaning it expects the return to be equally impacted by the price. In reality investors consider the upside potential when buying stocks and other securites. Investors will buy volitile stocks with the potential of higher returns. An investor may ignore the ratio and sell, just so they can cut their loses. There are other variations of the Sharpe Ratio called Sortino Ratio that ignores the above-average returns and focus on the downside deviation to handle risk.

https://www.investopedia.com/terms/s/sharperatio.asp

https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/overview

https://medium.com/codex/algorithmic-trading-with-relative-strength-index-in-python-d969cf22dd85

https://www.investopedia.com/terms/m/macd.asp

https://www.educba.com/moving-average-formula/

https://www.alphacodingskills.com/pandas/notes/pandas-function-dataframe-pct-change.php

https://www.kaggle.com/code/onurkoc83/technical-features-trading-strategy

https://www.kaggle.com/code/ikeppyo/examples-of-higher-scores-than-perfect-predictions